#**ANALYZING FOOD INDUSTRY IN QUEBEC**
###Known for its delicious multi-cultural cuisines, Quebec food industry has a huge potential for business expansion. I aim here to contribute towards this prospect and begin by scraping a wikipedia page containing list of all towns in Quebec province. I utilize the gathered data to understand the Food Business Sector and put forth possible suggestions for invesment to expand the industry. 

###I have used unsupervised clustering: K-Means for data visualization and analysis of various clusters. It achieves successful grouping of data points into three non-overlapping clusters which makes it one of the most credulous clustering algorithms.    



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests
import io
import folium
import geopy
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

#### Let' scrape a wikipedia(which needs serious donations, please donate) page to get the list of towns in Quebec

![Wikipedia logo](https://upload.wikimedia.org/wikipedia/commons/7/77/Wikipedia_svg_logo.svg)

In [ ]:
tables=pd.read_html("https://en.wikipedia.org/wiki/List_of_towns_in_Quebec")


In [ ]:
tables1=tables[1]
tables1

,Name,Regional county municipality,Region,CMA/CA,Population(2016)[2],Population(2011)[2],Change(%)[2],Area(km²)[2],Populationdensity[2]
0,Acton Vale,Acton,Montérégie,NaN,7656,7664,−0.1,91.10,84.0
1,Alma,Lac-Saint-Jean-Est,Saguenay–Lac-Saint-Jean,Alma,30776,30904,−0.4,196.54,156.6
2,Amos,Abitibi,Abitibi-Témiscamingue,Amos,12823,12671,1.2,430.29,29.8
3,Amqui,La Matapédia,Bas-Saint-Laurent,NaN,6178,6322,−2.3,121.17,51.0
4,Baie-Comeau,Manicouagan,Côte-Nord,Baie-Comeau,21536,22113,−2.6,336.59,64.0
...,...,...,...,...,...,...,...,...,...
220,Waterloo,La Haute-Yamaska,Montérégie,NaN,4410,4330,1.8,12.24,360.2
221,Waterville,Coaticook,Estrie,Sherbrooke,2121,2028,4.6,44.10,48.1
222,Westmount,NaN,Montréal,Montréal,20312,19931,1.9,4.04,5024.9
223,Windsor,Le Val-Saint-François,Estrie,NaN,5419,5330,1.7,14.56,372.2


In [ ]:
df_town=tables1.loc[:,['Name','Region']]
df_town

,Name,Region
0,Acton Vale,Montérégie
1,Alma,Saguenay–Lac-Saint-Jean
2,Amos,Abitibi-Témiscamingue
3,Amqui,Bas-Saint-Laurent
4,Baie-Comeau,Côte-Nord
...,...,...
220,Waterloo,Montérégie
221,Waterville,Estrie
222,Westmount,Montréal
223,Windsor,Estrie


In [ ]:
tables1.isna().any().any() #missing values present or not

True

In [ ]:
tables1.isna().sum().sum() #total no of missing values

148

In [ ]:
tables1.isna().sum()/(len(tables1))*100 #missing values percentage

Name                             0.000000
Regional county municipality    16.444444
Region                           0.000000
CMA/CA                          49.333333
Population(2016)[2]              0.000000
Population(2011)[2]              0.000000
Change(%)[2]                     0.000000
Area(km²)[2]                     0.000000
Populationdensity[2]             0.000000
dtype: float64

In [ ]:
tables1.loc[:, tables1.isnull().any()].columns #determining columns with missing values

Index(['Regional county municipality', 'CMA/CA'], dtype='object')

In [ ]:
df_town.isna().sum()/(len(df_town))*100 #no of missing values in df_town

Name      0.0
Region    0.0
dtype: float64

In [ ]:
#determining values with [] in Name column
print(df_town[df_town.Name.str.endswith("]")])
#rows with ___ in Region column
print(df_town.loc[215:224 , :])
df_town=df_town.drop([81,96,116,141,224])

                   Name              Region
81       La Tuque[QC 1]            Mauricie
96   L'Île-Dorval[QC 2]            Montréal
116      Montreal[QC 3]            Montréal
141        Québec[QC 4]  Capitale-Nationale
                 Name                 Region
215          Varennes             Montérégie
216  Vaudreuil-Dorion             Montérégie
217     Victoriaville       Centre-du-Québec
218       Ville-Marie  Abitibi-Témiscamingue
219           Warwick       Centre-du-Québec
220          Waterloo             Montérégie


In [ ]:
#reorder index after removing columns
df_town2=df_town
df_town2.reset_index(drop=True, inplace=True) 
df_town2.loc[78:220 , :]

,Name,Region
78,La Pocatière,Bas-Saint-Laurent
79,La Prairie,Montérégie
80,La Sarre,Abitibi-Témiscamingue
81,Lac-Delage,Capitale-Nationale
82,Lachute,Laurentides
...,...,...
215,Warwick,Centre-du-Québec
216,Waterloo,Montérégie
217,Waterville,Estrie
218,Westmount,Montréal


#### Let's get the geographical location of the towns


In [ ]:
count=df_town2.index
print(count)

RangeIndex(start=0, stop=220, step=1)


In [ ]:
#FOR FULL DATA (220 ROWS)
town_latitude=[0 for a in range(220)]
town_longitude=[0 for a in range(220)]
for i in list(range(220)):
  address = df_town2.loc[i,"Name"]+", "+df_town2.loc[i,"Region"]+", Quebec"
  #address
  locator = Nominatim(user_agent="quebec_explore")
  #locator
  location = locator.geocode(address)
  #location
  town_latitude[i]=location.latitude
  town_longitude[i]=location.longitude

print(town_latitude)
print(town_longitude)

[45.6471564, 48.548887, 48.5718519, 48.4656706, 49.217597, 45.4174258, 47.444343, 46.0005197, 45.4289766, 46.210725, 45.3139778, 47.043926, 46.339343, 45.120537, 47.3905166, 45.5643183, 46.0828098, 45.67932, 45.6130352, 45.6713243, 48.0444351, 45.6050197, 45.219192, 45.318116, 45.4554829, 45.676516, 45.3867947, 49.0959673, 46.6701717, 45.437955, 48.1014497, 48.3542375, 45.4473898, 48.3485676, 49.783855, 45.725519649999995, 45.3619066, 46.971117, 49.9137407, 47.694512, 45.1334169, 45.8573781, 45.4124593, 45.4772716, 45.2963957, 45.2076125, 45.785278, 46.20431, 47.547799, 45.3749509, 48.417809, 45.5372581, 45.904822, 48.883231, 45.48423, 46.676532, 45.4453082, 45.8864303, 45.130469, 48.5026134, 45.4828558, 46.0423647, 45.2831, 52.7931065, 48.739437, 46.871928, 48.6580556, 45.4841214, 46.0933721, 45.3990409, 48.5347749, 45.4811545, 45.464465, 45.087339, 46.0239798, 45.857978, 45.4529194, 47.657525, 47.369524, 45.4175472, 48.8000956, 46.9706256, 45.6560478, 45.5905023, 46.9361368, 46.85854

In [ ]:
df_town2['Latitude']=town_latitude
df_town2

,Name,Region,Latitude
0,Acton Vale,Montérégie,45.647156
1,Alma,Saguenay–Lac-Saint-Jean,48.548887
2,Amos,Abitibi-Témiscamingue,48.571852
3,Amqui,Bas-Saint-Laurent,48.465671
4,Baie-Comeau,Côte-Nord,49.217597
...,...,...,...
215,Warwick,Centre-du-Québec,45.945659
216,Waterloo,Montérégie,45.344157
217,Waterville,Estrie,45.277061
218,Westmount,Montréal,45.485733


In [ ]:
df_town2['Longitude']=town_longitude
df_town2

,Name,Region,Latitude,Longitude
0,Acton Vale,Montérégie,45.647156,-72.565411
1,Alma,Saguenay–Lac-Saint-Jean,48.548887,-71.651459
2,Amos,Abitibi-Témiscamingue,48.571852,-78.116086
3,Amqui,Bas-Saint-Laurent,48.465671,-67.431517
4,Baie-Comeau,Côte-Nord,49.217597,-68.152313
...,...,...,...,...
215,Warwick,Centre-du-Québec,45.945659,-71.990662
216,Waterloo,Montérégie,45.344157,-72.516217
217,Waterville,Estrie,45.277061,-71.890755
218,Westmount,Montréal,45.485733,-73.596395


#### Now let's plot the cities in a map of Canada

In [ ]:
Quebec_lat=48.571852
Quebec_lng=-79.201022

map_Quebec = folium.Map(location=[Quebec_lat, Quebec_lng], zoom_start=6)

for lat, lng, town, reg in zip(df_town2['Latitude'], df_town2['Longitude'], df_town2['Name'],df_town2['Region']):
    label='{}, {}'.format(town, reg)
    folium.Marker(location=[lat,lng]).add_to(map_Quebec)

map_Quebec

In [ ]:
CLIENT_ID = '*****'
CLIENT_SECRET = '*****'
VERSION='20180325'
LIMIT=5000
RADIUS=5000

import json

In [ ]:
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&limit={}&radius={}'.format(CLIENT_ID,CLIENT_SECRET,df_town2.loc[0,"Latitude"],
                                                                                            df_town2.loc[0,"Longitude"],VERSION,LIMIT,RADIUS)

In [ ]:
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5fe70fb08028c13d497ee507'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4cba25efc7228cfa3a3b15ce-0',
      'venue': {'beenHere': {'count': 0,
        'lastCheckinExpiredAt': 0,
        'marked': False,
        'unconfirmedCount': 0},
       'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1c5941735',
         'name': 'Sandwich Place',
         'pluralName': 'Sandwich Places',
         'primary': True,
         'shortName': 'Sandwiches'}],
       'contact': {},
       'hereNow': {'count': 0, 'groups': [], 'summary': 'Nobody here'},
       'id': '4cba25efc7228cfa3a3b15ce',
       'location': {'address': '690 Rue de Roxton',
        'cc': 'CA',
        'city': 'Acton Vale',
        

In [ ]:
LIMIT=400
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        try:
            results=requests.get(url).json()['response']['groups'][0]['items']
        except:
            continue;
        venues_list.append([(
        name, 
        lat, 
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Let's get all the venues in the towns of Quebec

In [ ]:
Quebec_venues = getNearbyVenues(names=df_town2['Name'],
                                   latitudes=df_town2['Latitude'],
                                   longitudes=df_town2['Longitude']
                                  )

Acton Vale
Alma
Amos
Amqui
Baie-Comeau
Baie-D'Urfé
Baie-Saint-Paul
Barkmere
Beaconsfield
Beauceville
Beauharnois
Beaupré
Bécancour
Bedford
Belleterre
Beloeil
Berthierville
Blainville
Boisbriand
Bois-des-Filion
Bonaventure
Boucherville
Lac-Brome
Bromont
Brossard
Brownsburg-Chatham
Candiac
Cap-Chat
Cap-Santé
Carignan
Carleton-sur-Mer
Causapscal
Chambly
Chandler
Chapais
Charlemagne
Châteauguay
Château-Richer
Chibougamau
Clermont
Coaticook
Contrecoeur
Cookshire-Eaton
Côte Saint-Luc
Coteau-du-Lac
Cowansville
Danville
Daveluyville
Dégelis
Delson
Desbiens
Deux-Montagnes
Disraeli
Dolbeau-Mistassini
Dollard-des-Ormeaux
Donnacona
Dorval
Drummondville
Dunham
Duparquet
East Angus
Estérel
Farnham
Fermont
Forestville
Fossambault-sur-le-Lac
Gaspé
Gatineau
Gracefield
Granby
Grande-Rivière
Hampstead
Hudson
Huntingdon
Joliette
Kingsey Falls
Kirkland
La Malbaie
La Pocatière
La Prairie
La Sarre
Lac-Delage
Lachute
Lac-Mégantic
Lac-Saint-Joseph
Lac-Sergent
L'Ancienne-Lorette
L'Assomption
Laval
Lavaltrie
Leb

In [ ]:
Quebec_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Acton Vale,45.647156,-72.565411,McDonald's,45.644022,-72.571456,Fast Food Restaurant
1,Acton Vale,45.647156,-72.565411,La Galoche,45.648941,-72.566303,Pub
2,Acton Vale,45.647156,-72.565411,IGA - Ernest Joly & Fils Inc.,45.645591,-72.574484,Grocery Store
3,Acton Vale,45.647156,-72.565411,Arena Acton Vale,45.655991,-72.573635,Skating Rink
4,Alma,48.548887,-71.651459,Le Café du Clocher,48.551003,-71.651355,Gastropub
...,...,...,...,...,...,...,...
3816,Windsor,45.569780,-72.003495,Tim Hortons,45.567284,-72.011410,Coffee Shop
3817,Windsor,45.569780,-72.003495,Arena J.A Lemay,45.573221,-71.993129,Hockey Arena
3818,Windsor,45.569780,-72.003495,Les Jardins Windsor,45.577835,-72.011509,Garden Center
3819,Windsor,45.569780,-72.003495,Construction-Rénovation Jean-François Roy Inc,45.562258,-71.992090,Construction & Landscaping


In [ ]:
Quebec_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Acton Vale,4,4,4,4,4,4
Alma,22,22,22,22,22,22
Amos,17,17,17,17,17,17
Amqui,8,8,8,8,8,8
Baie-Comeau,6,6,6,6,6,6
...,...,...,...,...,...,...
Warwick,7,7,7,7,7,7
Waterloo,4,4,4,4,4,4
Waterville,5,5,5,5,5,5


In [ ]:
Quebec_venues.groupby("Venue Category").count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,5,5,5,5,5,5
Accessories Store,3,3,3,3,3,3
Airport,2,2,2,2,2,2
Airport Lounge,4,4,4,4,4,4
Airport Service,3,3,3,3,3,3
...,...,...,...,...,...,...
Wine Shop,3,3,3,3,3,3
Wings Joint,2,2,2,2,2,2
Women's Store,2,2,2,2,2,2


In [ ]:
Quebec_onehot=pd.get_dummies(Quebec_venues[['Venue Category']], prefix="", prefix_sep="")
Quebec_onehot.drop("Neighborhood", axis=1, inplace=True)
Quebec_onehot['Neighborhood']=Quebec_venues['Neighborhood']

fixed_columns=[Quebec_onehot.columns[-1]]+list(Quebec_onehot.columns[:-1])
Quebec_onehot=Quebec_onehot[fixed_columns]
Quebec_onehot

,Neighborhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,...,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Transportation Service,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Acton Vale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Acton Vale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Acton Vale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Acton Vale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alma,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3816,Windsor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3817,Windsor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3818,Windsor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3819,Windsor,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
Quebec_grouped=Quebec_onehot.groupby("Neighborhood").mean().reset_index()
Quebec_grouped

,Neighborhood,ATM,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,...,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Trail,Train Station,Transportation Service,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Acton Vale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
1,Alma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.045455,0.0,0.000000,0.00,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.045455,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
2,Amos,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.058824,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
3,Amqui,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
4,Baie-Comeau,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Warwick,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.142857,0.00,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
210,Waterloo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.00,0.00,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.250000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0
211,Waterv

In [ ]:
def top_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Let's find the top 10 venues in every neighborhood

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Quebec_grouped['Neighborhood']

for ind in np.arange(Quebec_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = top_common_venues(Quebec_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Vale,Pub,Grocery Store,Skating Rink,Fast Food Restaurant,Zoo,Fish & Chips Shop,Farmers Market,Filipino Restaurant,Fireworks Store,Flea Market
1,Alma,Fast Food Restaurant,Pharmacy,Restaurant,Grocery Store,Coffee Shop,Convenience Store,City,Snack Place,Gastropub,Supermarket
2,Amos,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Convenience Store,Rental Car Location,Sandwich Place,Hardware Store,Gastropub,Pharmacy
3,Amqui,Food,Gas Station,Fast Food Restaurant,Construction & Landscaping,Hostel,Bar,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market
4,Baie-Comeau,Lounge,Motel,Restaurant,Brewery,Coffee Shop,Hotel,Football Stadium,Food Service,Food Court,Food & Drink Shop
...,...,...,...,...,...,...,...,...,...,...,...
209,Warwick,BBQ Joint,Grocery Store,Ice Cream Shop,Construction & Landscaping,Sandwich Place,Café,Restaurant,Fireworks Store,Farmers Market,Fast Food Restaurant
210,Waterloo,Liquor Store,Supermarket,Coffee Shop,Music Venue,Football Stadium,Food Service,French Restaurant,Food Court,Fried Chicken Joint,Farm
211,Waterville,Campground,Shipping Store,Grocery Store,Fireworks Store,Golf Course,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Zoo
212,Westmount,Café,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Yoga Studio,Park,Asian Restaurant,French Restaurant,Office


#### Let's cluster the areas

In [ ]:
kclusters = 3

Quebec_grouped_clustering = Quebec_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Quebec_grouped_clustering)

dtype('int32')

In [ ]:
df_town2.drop('Region',axis=1,inplace=True)

In [ ]:
df_town2.rename(columns={"Name":"Neighborhood"}, inplace=True)
df_town2

,Neighborhood,Latitude,Longitude
0,Acton Vale,45.647156,-72.565411
1,Alma,48.548887,-71.651459
2,Amos,48.571852,-78.116086
3,Amqui,48.465671,-67.431517
4,Baie-Comeau,49.217597,-68.152313
...,...,...,...
215,Warwick,45.945659,-71.990662
216,Waterloo,45.344157,-72.516217
217,Waterville,45.277061,-71.890755
218,Westmount,45.485733,-73.596395


In [ ]:
neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)

#### Let's merge the main dataset with the most common venues

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_.astype('int32'))
neighborhoods_venues_sorted["Cluster Labels"].dtype
Quebec_merged = df_town2

Quebec_merged = Quebec_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
print(Quebec_merged.shape)

(220, 14)


In [ ]:
Quebec_merged.reset_index(inplace=True)
Quebec_merged.drop("index",inplace=True, axis=1)
Quebec_merged

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Vale,45.647156,-72.565411,0.0,Pub,Grocery Store,Skating Rink,Fast Food Restaurant,Zoo,Fish & Chips Shop,Farmers Market,Filipino Restaurant,Fireworks Store,Flea Market
1,Alma,48.548887,-71.651459,2.0,Fast Food Restaurant,Pharmacy,Restaurant,Grocery Store,Coffee Shop,Convenience Store,City,Snack Place,Gastropub,Supermarket
2,Amos,48.571852,-78.116086,0.0,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Convenience Store,Rental Car Location,Sandwich Place,Hardware Store,Gastropub,Pharmacy
3,Amqui,48.465671,-67.431517,2.0,Food,Gas Station,Fast Food Restaurant,Construction & Landscaping,Hostel,Bar,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market
4,Baie-Comeau,49.217597,-68.152313,2.0,Lounge,Motel,Restaurant,Brewery,Coffee Shop,Hotel,Football Stadium,Food Service,Food Court,Food & Drink Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Warwick,45.945659,-71.990662,2.0,BBQ Joint,Grocery Store,Ice Cream Shop,Construction & Landscaping,Sandwich Place,Café,Restaurant,Fireworks Store,Farmers Market,Fast Food Restaurant
216,Waterloo,45.344157,-72.516217,2.0,Liquor Store,Supermarket,Coffee Shop,Music Venue,Football Stadium,Food Service,French Restaurant,Food Court,Fried Chicken Joint,Farm
217,Waterville,45.277061,-71.890755,0.0,Campground,Shipping Store,Grocery Store,Fireworks Store,Golf Course,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Zoo
218,Westmount,45.485733,-73.596395,2.0,Café,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Yoga Studio,Park,Asian Restaurant,French Restaurant,Office


In [ ]:
Quebec_merged.dropna()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Vale,45.647156,-72.565411,0.0,Pub,Grocery Store,Skating Rink,Fast Food Restaurant,Zoo,Fish & Chips Shop,Farmers Market,Filipino Restaurant,Fireworks Store,Flea Market
1,Alma,48.548887,-71.651459,2.0,Fast Food Restaurant,Pharmacy,Restaurant,Grocery Store,Coffee Shop,Convenience Store,City,Snack Place,Gastropub,Supermarket
2,Amos,48.571852,-78.116086,0.0,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Convenience Store,Rental Car Location,Sandwich Place,Hardware Store,Gastropub,Pharmacy
3,Amqui,48.465671,-67.431517,2.0,Food,Gas Station,Fast Food Restaurant,Construction & Landscaping,Hostel,Bar,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market
4,Baie-Comeau,49.217597,-68.152313,2.0,Lounge,Motel,Restaurant,Brewery,Coffee Shop,Hotel,Football Stadium,Food Service,Food Court,Food & Drink Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,Warwick,45.945659,-71.990662,2.0,BBQ Joint,Grocery Store,Ice Cream Shop,Construction & Landscaping,Sandwich Place,Café,Restaurant,Fireworks Store,Farmers Market,Fast Food Restaurant
216,Waterloo,45.344157,-72.516217,2.0,Liquor Store,Supermarket,Coffee Shop,Music Venue,Football Stadium,Food Service,French Restaurant,Food Court,Fried Chicken Joint,Farm
217,Waterville,45.277061,-71.890755,0.0,Campground,Shipping Store,Grocery Store,Fireworks Store,Golf Course,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Zoo
218,Westmount,45.485733,-73.596395,2.0,Café,Bakery,Coffee Shop,Pizza Place,Breakfast Spot,Yoga Studio,Park,Asian Restaurant,French Restaurant,Office


In [ ]:
Quebec_merged = Quebec_merged[~Quebec_merged['Cluster Labels'].isnull()]
Quebec_merged.shape

(214, 14)

#### Now let's plot the areas in map with a colour assigned to each cluster

In [ ]:
latitude=45.647156
longitude=-72.565411

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

arg = np.arange(kclusters)
rg = [i + x + (i*arg)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(rg)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, ngh, cluster in zip(Quebec_merged['Latitude'], Quebec_merged['Longitude'], Quebec_merged['Neighborhood'], Quebec_merged['Cluster Labels']):
    label = folium.Popup(str(ngh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color=rainbow[int(cluster)-1],
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
cluster_1=Quebec_merged[Quebec_merged.loc[:,"Cluster Labels"]==0.0].reset_index(drop=True)
cluster_1

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acton Vale,45.647156,-72.565411,0.0,Pub,Grocery Store,Skating Rink,Fast Food Restaurant,Zoo,Fish & Chips Shop,Farmers Market,Filipino Restaurant,Fireworks Store,Flea Market
1,Amos,48.571852,-78.116086,0.0,Grocery Store,Fast Food Restaurant,Gas Station,Coffee Shop,Convenience Store,Rental Car Location,Sandwich Place,Hardware Store,Gastropub,Pharmacy
2,Baie-D'Urfé,45.417426,-73.915364,0.0,Zoo,Train Station,Grocery Store,Sandwich Place,Liquor Store,Hotel,Diner,Flea Market,Department Store,Fast Food Restaurant
3,Beauharnois,45.313978,-73.875834,0.0,Fast Food Restaurant,Grocery Store,Hockey Arena,Sandwich Place,Gas Station,Business Service,Coffee Shop,Zoo,Filipino Restaurant,Fireworks Store
4,Bedford,45.120537,-72.988701,0.0,Coffee Shop,Pub,Grocery Store,Hot Dog Joint,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop
5,Belleterre,47.390517,-78.704297,0.0,Liquor Store,French Restaurant,Grocery Store,Flea Market,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Zoo
6,Berthierville,46.082810,-73.174710,0.0,Grocery Store,Fast Food Restaurant,Restaurant,Liquor Store,Coffee Shop,Museum,Gas Station,Pharmacy,Fried Chicken Joint,Hotel
7,Chibougamau,49.913741,-74.371395,0.0,Construction & Landscaping,Grocery Store,Fast Food Restaurant,Sandwich Place,Zoo,Flea Market,Farmers Market,Filipino Restaurant,Fireworks Store,Fish & Chips Shop
8,Clermont,47.694512,-70.226220,0.0,Convenience Store,Skating Rink,Grocery Store,Rental Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Falafel Restaurant
9,Contrecoeur,45.857378,-73.237431,0.0,Gas Station,Harbor / Marina,Grocery Store,Flea Market,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Zoo


In [ ]:
cluster_2=Quebec_merged[Quebec_merged.loc[:,"Cluster Labels"]==1.0].reset_index(drop=True)
cluster_2

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barkmere,46.000520,-74.574633,1.0,Lake,Zoo,Farm,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market,Flower Shop,Food
1,Beauceville,46.210725,-70.774269,1.0,Construction & Landscaping,Grocery Store,Sandwich Place,Coffee Shop,Electronics Store,Zoo,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store
2,Bécancour,46.339343,-72.433205,1.0,Construction & Landscaping,Gas Station,Gastropub,Hockey Rink,Arts & Crafts Store,Zoo,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Fireworks Store
3,Brownsburg-Chatham,45.676516,-74.404606,1.0,ATM,Construction & Landscaping,Hotel,Burger Joint,Flower Shop,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market
4,Cap-Chat,49.095967,-66.687883,1.0,Construction & Landscaping,Diner,Gas Station,Pharmacy,Zoo,Food,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market
5,Carignan,45.437955,-73.355490,1.0,Construction & Landscaping,Health & Beauty Service,Home Service,Zoo,Flea Market,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop
6,Daveluyville,46.204310,-72.137604,1.0,Construction & Landscaping,Flower Shop,Pharmacy,Zoo,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market,Food
7,Desbiens,48.417809,-71.949463,1.0,Home Service,Campground,Flower Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market,Zoo
8,Fossambault-sur-le-Lac,46.871928,-71.618500,1.0,Construction & Landscaping,Home Service,Spa,Gym / Fitness Center,Food,Golf Course,Snack Place,Park,Restaurant,Fast Food Restaurant
9,Huntingdon,45.087339,-74.173851,1.0,Construction & Landscaping,Grocery Store,Hot Dog Joint,Sandwich Place,Restaurant,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [ ]:
cluster_3=Quebec_merged[Quebec_merged.loc[:,"Cluster Labels"]==2.0].reset_index(drop=True)
cluster_3

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alma,48.548887,-71.651459,2.0,Fast Food Restaurant,Pharmacy,Restaurant,Grocery Store,Coffee Shop,Convenience Store,City,Snack Place,Gastropub,Supermarket
1,Amqui,48.465671,-67.431517,2.0,Food,Gas Station,Fast Food Restaurant,Construction & Landscaping,Hostel,Bar,Filipino Restaurant,Fireworks Store,Fish & Chips Shop,Flea Market
2,Baie-Comeau,49.217597,-68.152313,2.0,Lounge,Motel,Restaurant,Brewery,Coffee Shop,Hotel,Football Stadium,Food Service,Food Court,Food & Drink Shop
3,Baie-Saint-Paul,47.444343,-70.505447,2.0,Hotel,Restaurant,French Restaurant,Ice Cream Shop,Brewery,Bakery,Breakfast Spot,Spa,Diner,Museum
4,Beaconsfield,45.428977,-73.865439,2.0,Pizza Place,Pharmacy,Bank,Pub,Train Station,Sporting Goods Shop,Shopping Mall,Gas Station,Supermarket,Liquor Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,Victoriaville,46.057370,-71.953667,2.0,Coffee Shop,Fast Food Restaurant,Restaurant,Grocery Store,Brewery,Bar,Mexican Restaurant,Gastropub,Health Food Store,Gas Station
142,Ville-Marie,47.332308,-79.438019,2.0,Department Store,Hotel,Liquor Store,Gas Station,Pharmacy,Zoo,Flea Market,Fast Food Restaurant,Filipino Restaurant,Fireworks Store
143,Warwick,45.945659,-71.990662,2.0,BBQ Joint,Grocery Store,Ice Cream Shop,Construction & Landscaping,Sandwich Place,Café,Restaurant,Fireworks Store,Farmers Market,Fast Food Restaurant
144,Waterloo,45.344157,-72.516217,2.0,Liquor Store,Supermarket,Coffee Shop,Music Venue,Football Stadium,Food Service,French Restaurant,Food Court,Fried Chicken Joint,Farm


### Let's analyze the two clusters

In [ ]:
cluster_1.groupby("1st Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1st Most Common Venue,,,,,,,,,,,,,
Grocery Store,15,15,15,15,15,15,15,15,15,15,15,15,15
Fast Food Restaurant,4,4,4,4,4,4,4,4,4,4,4,4,4
Liquor Store,4,4,4,4,4,4,4,4,4,4,4,4,4
Construction & Landscaping,3,3,3,3,3,3,3,3,3,3,3,3,3
Pharmacy,3,3,3,3,3,3,3,3,3,3,3,3,3
Gas Station,3,3,3,3,3,3,3,3,3,3,3,3,3
Hardware Store,2,2,2,2,2,2,2,2,2,2,2,2,2
Hockey Arena,2,2,2,2,2,2,2,2,2,2,2,2,2
Pub,2,2,2,2,2,2,2,2,2,2,2,2,2


#### The most frequent "1st most common venue in first cluster is Grocery Store" followed by "Fast Food restaurant" and "Liquor Store". This is also the similar to findings in 2nd most common venues too

In [ ]:
cluster_1.groupby("2nd Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2nd Most Common Venue,,,,,,,,,,,,,
Grocery Store,12,12,12,12,12,12,12,12,12,12,12,12,12
Fast Food Restaurant,5,5,5,5,5,5,5,5,5,5,5,5,5
Gas Station,3,3,3,3,3,3,3,3,3,3,3,3,3
Hockey Arena,3,3,3,3,3,3,3,3,3,3,3,3,3
Sandwich Place,2,2,2,2,2,2,2,2,2,2,2,2,2
Department Store,2,2,2,2,2,2,2,2,2,2,2,2,2
Pharmacy,2,2,2,2,2,2,2,2,2,2,2,2,2
ATM,1,1,1,1,1,1,1,1,1,1,1,1,1
Hockey Field,1,1,1,1,1,1,1,1,1,1,1,1,1


#### Gym and Fitness Center and shopping malls are also popular in cluster 1

In [ ]:
cluster_1.groupby("6th Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6th Most Common Venue,,,,,,,,,,,,,
Fast Food Restaurant,7,7,7,7,7,7,7,7,7,7,7,7,7
Farmers Market,6,6,6,6,6,6,6,6,6,6,6,6,6
Flea Market,5,5,5,5,5,5,5,5,5,5,5,5,5
Zoo,3,3,3,3,3,3,3,3,3,3,3,3,3
Fish & Chips Shop,3,3,3,3,3,3,3,3,3,3,3,3,3
Flower Shop,2,2,2,2,2,2,2,2,2,2,2,2,2
Gas Station,2,2,2,2,2,2,2,2,2,2,2,2,2
Pharmacy,2,2,2,2,2,2,2,2,2,2,2,2,2
Museum,1,1,1,1,1,1,1,1,1,1,1,1,1


#### Fireworks Store and Fish & Chips Shops seems to have taken 10th most frequent available places in Quebec

### Now let's see the most frequent top venues in the second cluster

In [ ]:
cluster_2.groupby("1st Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1st Most Common Venue,,,,,,,,,,,,,
Construction & Landscaping,13,13,13,13,13,13,13,13,13,13,13,13,13
ATM,2,2,2,2,2,2,2,2,2,2,2,2,2
Ice Cream Shop,2,2,2,2,2,2,2,2,2,2,2,2,2
Convenience Store,1,1,1,1,1,1,1,1,1,1,1,1,1
Home Service,1,1,1,1,1,1,1,1,1,1,1,1,1
Hotel,1,1,1,1,1,1,1,1,1,1,1,1,1
Lake,1,1,1,1,1,1,1,1,1,1,1,1,1


###### Construction and Landscaping, ATMs are the also among the available venues in cluster 2

#### Filipino restaurants are also preferable venue in cluster 2

In [ ]:
cluster_2.groupby("10th Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
10th Most Common Venue,,,,,,,,,,,,,
Fish & Chips Shop,6,6,6,6,6,6,6,6,6,6,6,6,6
Filipino Restaurant,3,3,3,3,3,3,3,3,3,3,3,3,3
Fireworks Store,3,3,3,3,3,3,3,3,3,3,3,3,3
Flea Market,3,3,3,3,3,3,3,3,3,3,3,3,3
Food,3,3,3,3,3,3,3,3,3,3,3,3,3
Fast Food Restaurant,1,1,1,1,1,1,1,1,1,1,1,1,1
Hotel Bar,1,1,1,1,1,1,1,1,1,1,1,1,1
Zoo,1,1,1,1,1,1,1,1,1,1,1,1,1


#### Hotel Bar and Zoo are also among the most available venues in cluster 2

Now let's see the most frequent top venues in the third cluster

In [ ]:
cluster_3.groupby("1st Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1st Most Common Venue,,,,,,,,,,,,,
Fast Food Restaurant,26,26,26,26,26,26,26,26,26,26,26,26,26
Coffee Shop,19,19,19,19,19,19,19,19,19,19,19,19,19
Restaurant,12,12,12,12,12,12,12,12,12,12,12,12,12
Pharmacy,11,11,11,11,11,11,11,11,11,11,11,11,11
Hotel,9,9,9,9,9,9,9,9,9,9,9,9,9
Grocery Store,7,7,7,7,7,7,7,7,7,7,7,7,7
Convenience Store,6,6,6,6,6,6,6,6,6,6,6,6,6
Gas Station,5,5,5,5,5,5,5,5,5,5,5,5,5
Bar,4,4,4,4,4,4,4,4,4,4,4,4,4


In [ ]:
cluster_3.groupby("10th Most Common Venue").count().sort_values(by="Neighborhood",ascending=False)

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue
10th Most Common Venue,,,,,,,,,,,,,
Fish & Chips Shop,14,14,14,14,14,14,14,14,14,14,14,14,14
Flower Shop,8,8,8,8,8,8,8,8,8,8,8,8,8
Supermarket,7,7,7,7,7,7,7,7,7,7,7,7,7
Flea Market,6,6,6,6,6,6,6,6,6,6,6,6,6
Grocery Store,5,5,5,5,5,5,5,5,5,5,5,5,5
Liquor Store,4,4,4,4,4,4,4,4,4,4,4,4,4
Filipino Restaurant,4,4,4,4,4,4,4,4,4,4,4,4,4
Fast Food Restaurant,4,4,4,4,4,4,4,4,4,4,4,4,4
Farmers Market,4,4,4,4,4,4,4,4,4,4,4,4,4


###### Farms and Bakery are among available places in cluster 3. Airport Terminal, Museum, Night Club and Hockey Arena are also some of the most common venues in Cluster 3.

## Final Suggestion: Any investment other than Fast Food restaurant, Grocery Store and Coffee Shop is better, considering there's a market for it. Bus Station, Shopping Malls and Beach are lagging behind in the frequency. So investing in them would make a better sense as there's a market for it but the availability is lesser.